In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [7]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [8]:
y_30sec_labels = train_lre_30sec["lang"]

In [9]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [10]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [11]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [12]:
#Shuffle the Dataset
#X_train_10sec_F,  X_train_30sec_F = shuffle(X_train_10sec, X_train_30sec, random_state=0)

In [13]:
import torch.nn as nn

In [14]:
#Implementing Generator as Encoder-Decoder pair inspired from DAGAN base paper

#Encoder should be able to take a batch of input(of dim 500-ivector) and be able to produce 
#its representation r

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(500, 512)
        self.bn_1 = torch.nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn_2 = torch.nn.BatchNorm1d(256)

        self.fc3 = nn.Linear(256, 128)
    def forward(self, x):
        return self.fc3(self.bn_2(F.relu(self.fc2(self.bn_1(F.relu(self.fc1(x)))))))
    
#We will use Decoder as Generator    
#Decoder should be able to take input of dim 128(r) and 100(z),concatenate r and z and 
#produce an output od dim 500-ivector    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(128,256 )
        self.bn_1 = torch.nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn_2 = torch.nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512,500)
    def forward(self, inputs):
        inputs = inputs #torch.cat([z, x], 1)
        return (self.fc3(self.bn_2(F.relu(self.fc2(self.bn_1(F.relu(self.fc1(inputs))))))))

#input to Generator alias Decoder is inputs = torch.cat([z, r], 1)
#Where r is the output of encoder given x i.e., representation of x encoded by encoder


In [15]:
#As we defined the Encoder and Decoder we now Implement the Generator 

class Generator(nn.Module):
                                
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, x):
        inputs = x   #torch.cat([z, x], 1)
        return self.fc2(self.fc1(inputs))

In [16]:
G = Generator()


In [17]:
G.cuda()

Generator(
  (fc1): Encoder(
    (fc1): Linear(in_features=500, out_features=512)
    (bn_1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
    (fc2): Linear(in_features=512, out_features=256)
    (bn_2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True)
    (fc3): Linear(in_features=256, out_features=128)
  )
  (fc2): Decoder(
    (fc1): Linear(in_features=128, out_features=256)
    (bn_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True)
    (fc2): Linear(in_features=256, out_features=512)
    (bn_2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
    (fc3): Linear(in_features=512, out_features=500)
  )
)

In [18]:
#G.modules

In [19]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [20]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
#D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [21]:
len(X_train_10sec)

114276

In [22]:
len(X_train_30sec)

114276

In [23]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train_10sec) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train_10sec[start:end]

        c = X_train_30sec[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))
        X = X.cuda()
        c = c.cuda()
        # Dicriminator forward-loss-backward-update
        G_sample = G(X)
        #D_real = D(X, c)
        #D_fake = D(G_sample, c)

        #D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        #D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        #D_loss = D_loss_real + D_loss_fake

        #D_loss.backward()
        #D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        #D_fake = D(G_sample, c)
        G_loss = criterion(G_sample, c)
        #G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()
        
        G_solver.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        #print('Iter-{}; G_loss: {}'.format(it, G_loss.data.numpy()))



        it+= batch_size
    print('G_loss: {}'.format(G_loss.cpu().data.numpy()))
           

epoch: 0
G_loss: [ 1.56981432]
epoch: 1
G_loss: [ 1.48842525]
epoch: 2
G_loss: [ 1.45837951]
epoch: 3
G_loss: [ 1.44716787]
epoch: 4
G_loss: [ 1.44149923]
epoch: 5
G_loss: [ 1.43748081]
epoch: 6
G_loss: [ 1.43446875]
epoch: 7
G_loss: [ 1.43198037]
epoch: 8
G_loss: [ 1.42990685]
epoch: 9
G_loss: [ 1.42809391]
epoch: 10
G_loss: [ 1.42652392]
epoch: 11
G_loss: [ 1.42498887]
epoch: 12
G_loss: [ 1.42354906]
epoch: 13
G_loss: [ 1.42217588]
epoch: 14
G_loss: [ 1.42079389]
epoch: 15
G_loss: [ 1.41930628]
epoch: 16
G_loss: [ 1.41789043]
epoch: 17
G_loss: [ 1.41641748]
epoch: 18
G_loss: [ 1.41502798]
epoch: 19
G_loss: [ 1.41341376]
epoch: 20
G_loss: [ 1.41176403]
epoch: 21
G_loss: [ 1.41001999]
epoch: 22
G_loss: [ 1.40831685]
epoch: 23
G_loss: [ 1.40647137]
epoch: 24
G_loss: [ 1.4046942]
epoch: 25
G_loss: [ 1.40297377]
epoch: 26
G_loss: [ 1.40122342]
epoch: 27
G_loss: [ 1.39950621]
epoch: 28
G_loss: [ 1.39791954]
epoch: 29
G_loss: [ 1.39639461]
epoch: 30
G_loss: [ 1.39487183]
epoch: 31
G_loss: [

In [24]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec))

In [25]:
gen_samples = G(X_train_30sec_gen.cuda())

In [26]:
gen_samples.data.shape

torch.Size([114276, 500])

In [27]:
#Taking the generated iVectors we will try to check the acc by MLP

In [28]:
train_X_gen = gen_samples.cpu().data.numpy()

In [29]:
type(train_X_gen)

numpy.ndarray

In [30]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [31]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [32]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [33]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [34]:
y_30sec_labels=le.transform(y_30sec_labels)

In [35]:
y_30sec_labels.shape

(114276,)

In [36]:
nb_classes = 14

In [37]:
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [38]:
Y_train_labels.shape

(114276, 14)

In [39]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_03Jan.csv')


In [40]:
eval_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/eval_feat_BNF_h5_03Jan.csv')


In [41]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [42]:
X_eval = eval_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_eval = eval_lre["language_code"]
y_eval_segmentid = eval_lre["segmentid1"]

In [43]:
X_val = X_val.values
X_eval = X_eval.values

In [44]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [45]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [46]:
y_val_labels = le.transform(y_val)

In [47]:
le = preprocessing.LabelEncoder()
le.fit(y_eval)

LabelEncoder()

In [48]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [49]:
y_eval_labels = le.transform(y_eval)

In [50]:
y_eval_labels[0:10]

array([12,  2,  5,  3,  3,  1, 12, 13,  6,  5])

In [51]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [52]:
Y_val

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [53]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                7182      
__________

In [55]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [56]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [57]:
batch_size = 256
nb_epoch=25

In [58]:
#train_X_gen
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X_gen,Y_train_labels , batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 114276 samples, validate on 3661 samples
Epoch 1/25
Epoch 00000: val_acc improved from -inf to 0.42884, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
9s - loss: 0.3259 - acc: 0.9009 - val_loss: 3.5983 - val_acc: 0.4288
Epoch 2/25
Epoch 00001: val_acc improved from 0.42884 to 0.48948, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
9s - loss: 0.1224 - acc: 0.9598 - val_loss: 3.2403 - val_acc: 0.4895
Epoch 3/25
Epoch 00002: val_acc did not improve
9s - loss: 0.1024 - acc: 0.9662 - val_loss: 3.3638 - val_acc: 0.4865
Epoch 4/25
Epoch 00003: val_acc improved from 0.48948 to 0.51352, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
9s - loss: 0.0898 - acc: 0.9702 - val_loss: 3.1103 - val_acc: 0.5135
Epoch 5/25
Epoch 00004: val_acc did not improve
9s - loss: 0.0816 - acc: 0.9723 - val_loss: 3.5352 - val_acc: 0.5007
Epoch 6/25
Epoch 00005: val_acc did not improve
9s - loss: 0.0754 - acc: 0.9741 - val_loss: 3.5787

In [59]:
score = model.evaluate(X_eval, Y_eval, verbose=0)
print('Acc %:', score[1])

Acc %: 0.526108993759
